# Prerequisites

## Install the Hazelcast Python Client

In [14]:
!pip install hazelcast-python-client==5.3.0

## Importing the Hazelcast Python Client

In [15]:
import hazelcast

## Importing classes

In [16]:
from models.person import Person
from models.address import Address

# Python Client

## Creating a Hazelcast Client

In [17]:
client = hazelcast.HazelcastClient(cluster_name="DB2", )

## Write Data to Memory

In [18]:
persons_map = client.get_map("persons").blocking()
addresses_map = client.get_map("addresses").blocking()

# Beispiel-Entitäten erstellen
person1 = Person("1", "John Doe")
address1 = Address("1", "Stuttgart", person1.id)
address2 = Address("2", "Berlin", person1.id)

# Entitäten in den Maps speichern
persons_map.put(person1.id, person1)
addresses_map.put(address1.id, address1)
addresses_map.put(address2.id, address2)


## Read Data from Memory

In [19]:
person_id_to_query = "1"
addresses_for_person = [address for address in addresses_map.values() if address.person_id == person_id_to_query]

print(f"Adressen für Person {person_id_to_query}: {[address.details for address in addresses_for_person]}")


Adressen für Person 1: ['Stuttgart', 'Berlin']


# Use SQL for Hazelcast in Python
## Create a SQL Table

In [20]:
sql = client.sql

# Tabelle erstellen (Map)
query = """
CREATE MAPPING cities 
__key INT,
countries VARCHAR,
cities VARCHAR)
TYPE IMap
OPTIONS('keyFormat'='int', 'valueFormat'='json-flat');
"""

sql.execute(query)

## Insert Data with SQL

In [21]:
# Daten in die Tabelle einfügen
query = """
INSERT INTO cities VALUES
(1, 'United Kingdom','London'),
(2, 'United Kingdom','Manchester'),
(3, 'United States', 'New York'),
(4, 'United States', 'Los Angeles'),
(5, 'Turkey', 'Ankara'),
(6, 'Turkey', 'Istanbul'),
(7, 'Brazil', 'Sao Paulo'),
(8, 'Brazil', 'Rio de Janeiro');
"""

sql.execute(query)


## Query Data with SQL

In [22]:
# Daten abfragen
query = "SELECT * FROM cities"
result = sql.execute(query).result()

# Ergebnisse durchgehen
for row in result:
    print(row)

HazelcastSqlError: Object 'cities' not found, did you forget to CREATE MAPPING?

## Query data with python client

In [23]:
# Daten abfragen
citiy_map = client.get_map("cities").blocking()

for key, value in citiy_map.entry_set():
    print(key, value)

# Example: Bookstore
This example demonstrates how to use Hazelcast to store and query data for a bookstore. The example uses three maps to store data for authors, books, and publishers. The example also demonstrates how to query the data.

In [27]:
import hazelcast

# Hazelcast-Client konfigurieren und starten
client = hazelcast.HazelcastClient(cluster_name="DB2", )

# Maps für die Entitäten erstellen
autoren_map = client.get_map("autoren").blocking()
buecher_map = client.get_map("buecher").blocking()
verlage_map = client.get_map("verlage").blocking()

# Autoren hinzufügen
autoren_map.put_if_absent("rowling", {"Name": "Rowling", "Vorname": "J.K.", "Alter": 57})
autoren_map.put_if_absent("collins", {"Name": "Collins", "Vorname": "Suzanne", "Alter": 60})
autoren_map.put_if_absent("tolkien", {"Name": "Tolkien", "Vorname": "J.R.R.", "Alter": 81})

# Verlage hinzufügen
verlage_map.put(1, {"Name": "Carlsen Verlag"})
verlage_map.put(2, {"Name": "Klett Cotta"})
verlage_map.put(3, {"Name": "Oetinger Verlag"})

# Bücher der Harry Potter-Serie
harry_potter_books = [
    {"ISBN": "9780747532699", "Name": "Harry Potter und der Stein der Weisen"},
    {"ISBN": "9780747538493", "Name": "Harry Potter und die Kammer des Schreckens"},
    {"ISBN": "9780747542155", "Name": "Harry Potter und der Gefangene von Askaban"},
    {"ISBN": "9780747546245", "Name": "Harry Potter und der Feuerkelch"},
    {"ISBN": "9780747551003", "Name": "Harry Potter und der Orden des Phoenix"},
    {"ISBN": "9780747581086", "Name": "Harry Potter und der Halbblutprinz"},
    {"ISBN": "9780747591054", "Name": "Harry Potter und die Heiligtuemer des Todes"}
]

# Bücher der Tribute von Panem-Serie
hunger_games_books = [
    {"ISBN": "9780439023481", "Name": "Die Tribute von Panem – Toedliche Spiele"},
    {"ISBN": "9780439023498", "Name": "Die Tribute von Panem – Gefaehrliche Liebe"},
    {"ISBN": "9780439023511", "Name": "Die Tribute von Panem – Flammender Zorn"}
]

# Bücher der Herr der Ringe-Serie
lord_of_the_rings_books = [
    {"ISBN": "9780261102354", "Name": "Der Herr der Ringe: Die Gefaehrten"},
    {"ISBN": "9780261102361", "Name": "Der Herr der Ringe: Die zwei Tuerme"},
    {"ISBN": "9780261102378", "Name": "Der Herr der Ringe: Die Rueckkehr des Koenigs"}
]

# Harry Potter Bücher hinzufügen
for book in harry_potter_books:
    buecher_map.put(book['ISBN'], {**book, "AutorID": "rowling", "VerlagID": 1})

# Tribute von Panem Bücher hinzufügen
for book in hunger_games_books:
    buecher_map.put(book['ISBN'], {**book, "AutorID": "collins", "VerlagID": 3})

# Herr der Ringe Bücher hinzufügen
for book in lord_of_the_rings_books:
    buecher_map.put(book['ISBN'], {**book, "AutorID": "tolkien", "VerlagID": 2})






## Query Data

In [28]:
sql = client.sql

In [32]:
# Alle Bücher von J.K. Rowling abfragen
result = [book["Name"] for book in buecher_map.values() if book["AutorID"] == "rowling"]

# # Mit SQL
# query = "SELECT * FROM buecher WHERE AutorID = 'rowling'"
# result = sql.execute(query).result()
    
# Ergebnisse durchgehen
for row in result:
    print(row)

Harry Potter und der Orden des Phoenix
Harry Potter und der Feuerkelch
Harry Potter und der Gefangene von Askaban
Harry Potter und die Heiligtuemer des Todes
Harry Potter und der Halbblutprinz
Harry Potter und der Stein der Weisen
Harry Potter und die Kammer des Schreckens


## Shutdown the Client

In [ ]:
# Verbindung zum Cluster beenden
client.shutdown()